In [138]:
from src.text_statistics.indexes import (
    coincidence_index,
    mutual_coincidence_index,
)
from src.text_statistics.сaesars_shift import caesars_shift
from src.text_statistics.statistics import xi_square
from src.text_statistics.grams import bigram_frequencies
from src.text_statistics.default_frequencies import BIGRAM_FREQUENCIES

In [139]:
FILE_PATH = r"D:\Pycharm-project\crypto_labs\data\lab4\2.TXT"

In [140]:
with open(FILE_PATH, encoding="utf-8") as file:
    text = file.read().split()

text = "".join(text)
text

'КГНГЖФЩРЩКЮКВМШЩЫЛНВНФЧЫФЬМНЫРУКЩУМЬНИКДЯЖЖТМБХСУГЫЦБЕРШЩЕЭНБЧТЭЖБЬЖЕЖВКИЧХФАЕЬМЮЛНЩИИГДДШЯМЩППЮЧТМЮКШДЖПВШГОЗСКЧТВЖЦЮУГВЭСФМГЯУТПЧШХЫГГЭЦКОКДМЦЧКВЛХОАСЫДВВМЖШИЖМЭЯДЭМАЯЮТЯЛГЖГЦШЛСЮЫТЖЬЯЧЧЫВМШЭСЭШВЖЦЮЧАЩЮХЩВИЩЧЛБУЦЩПЬБХРХУЮАРШДЮЕЬБУПЭПУЩЮАЧШГЖЦРВЩСЬОРШИГБЩХНОЭАОЩЭЫЮМЮПГАБВЮПИРЧФЮУЦЫЩЮШЦУААЯЛВЬШИЕЭПЦЧШЗЫБОШЬДАЬЭГЕЭЫХФШЧЫЖГЧВЕТУБИДНЖЩЬВИЫУЕЭЛКОГАЖЭУУНТЭВЯЯДКХЯЛЧРХВЫОЬМГУОДЭАЗВХЕЬНЭУРЬФШШЦЬЫВУЮОБКИЯШГТБХЧЧЯЫМГНМШИМЭКГЧИПЧИЗШГЬКТЖЮЖВЫФСКАЧАУШДВЛСЛЬМЕЭПЭТПЫЮСААЗПАВЮРВОПЮПЩЦУГБЩНПОЗВСЕЭПАОШИЦГШВРКОЭАЬЩОПЧЗФЕЩЯААВВОЩЗСШУНИЬЬЩФТУШЬДОЗТФБШТЯТЯВАСШКЖЫДЮЭМЖЕКВЦРЦНХЖЗГЯНЮЬСЖПММЬЬЩФЦФДГАОЗАОПТППКЭПНХБИГЯЕИЛИБЧПЮПЩЦУГАЕЮОУЫЕДВППППАПАЦЮТВЛАДЧУЭФБЩЬЩПЦЕЫЯЮЖРБКГДПХТТЭТОЮМЛЗЕЛРДСГЬКЭКСФАЦАЧЩООАЦГБУЖШКВФЩЮБАЦЛМЩОИАЮОНЩПЛБВЧЕЯРМЮЯХФЦКХХХДЗЩЯАДЛМЮДНЗЭПТЬВБПДЭЩХЗЬЕАФПРТЯДЯФЯЫШЖЕЛЧЛЧРЭКРЬШРЛОЫЫИРЕНЖХЗФМФШКУПЦВБРЗЭМЗАПЗЧКДЫЮНЦГОЫГМРЖКХЩЧКМЬЩФШМААЧШСЖЫСБЛФДЦРШУЯГАФЗЕЯТЭФКГЬЬЭФРЦЕЧЯЛИВЭГЕЫКБТЩЖДЩЫЕВЭСЬАЗЭЭТМУЧЖББТЫВЛИЭЬФЮЬКШТЭЖДЩЫЬЕРПЗЭБШРКЩКАЩЖСТАЛВСЛЧФЖНЖЩЦММЭБУЩХВСГЬЧЭЧБЩЬЮИКЦИАПВНБСБДУЕХБХЧЧЩЯЕЫКОЭЧОЭЭЫЩКЫ

# Дешифрование строится на основе индекса совпадений Фридмана

## Задаем алфавит

In [141]:
ALPHABET = "АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЬЫЭЮЯ"
ALPHABET_INDEX = 0.053

## Ищем размер ключа

In [142]:
for key_size in range(2, len(text) // 2):
    column = text[::key_size]

    if coincidence_index(column) >= ALPHABET_INDEX:
        break

key_size

17

## Востанавливаем ключ
Разбиение текст на столбцы шифрованных одним алфавитом

In [143]:
strings = [""] * key_size
for i, char in enumerate(text):
    strings[i % key_size] += char

strings

['КЛМЦЕИДГЭДТЧХУАЩВАШЧЭКАВМГДАЩВАШКЗДИЕТЯЛЧАЕАЩЖИРЫААЧЕЫЬАЩАЕЭТАДИКЖЭЖТАЖЭББДКИЧЦПГЩКНЯЩЕЭАРЕБААВПЕЧЩЦЧЫАМЭЦГЩДИВЧВЕЛЕИЕИДЗЕБПЕДИЗВАЗЕДЫГДЧЮЦЕИИЮТАЩЭГИЩКЖАЭЧЩЧКЧГКДАЕЕЧЧНЧБАИЦПЬЧУЖПБДАЩЯЧИЧСЧКЕЧЭЕИИВЕУБДЩИЗКЕЫЧЕЕУИРЧВЧЩИЫЕЩБЗИЭБКЧИВЭАЩЭОЯЕБЭЧЧДИКЕЗЕЧЭЕЧКТЧЖЕЫВЛДДЕЧЕАМ',
 'ГНЬБЖГЖВЦВЯЫЩЮЧХЮЯЬВЛХЗУГЬВЗНРВЬЖГГГЮВЮЗЩЦЯДХЕРЗГЧФЯВВЕЛХПЫЮЖКГРЬЕЗДОДГЖГХЬЮЯЧХЬХГЖДСЖЕЮЗРИГЬЛЮЬГНВЩЯВЩЕЮНЬГГЮФЖЬЬНБЗВГХХЖЕРШХАХХОЬБВВЬГДЬЯАДЖЬЖМГЖДЮЬЧЕНЩДСЭЖЕЬЮГЕЫЬВВОЕЕЧГЮЬЮЕЛЕХЮЮЬЬЧЖЮЭРЭХЧЧЗЧЬЖЬЕЮГГЕЯХЮШЬЖАЕАОГЛЮЦГГОЭГХБЕЮЮГДЗДБЭГИСЮАЕАВБЗЧАЮХЧЮЛЗЫЩГЗГФЯЮЯХКБЖЮЧХ',
 'НВНЕВДПЭКВЛВВАШНПЛДЕКЯВЮНКЛППКВДЫЯАЯОЛЖЕОЛРЛЗЛЕЭМШЗЛЭЛРВВВКЖВЕБВЗВКЭЕНЗВКЗПВЕЗЫЗЕЯЫЛАЛЛОВЛОКАЭЮИПКШКЛЕЛЭЕВКЕОИООДИЕШЛЯБЖЗПВМНФМИПЧОМЛЕКЯЛЮНЧЛРМЯЭНЗРЕБВЛВЭНАЛЯЭАТЕИРИКЭМЕВОАЖКОООЭОТФОБВЗЕЭОНННОВЭНЕЕВСНЕЛЛЗЯЕНЕЛЛЕЭВЭОЛНЖМББЖРЛПТБЛВНВЗБНЯЕЛЭБОЛЕЛЕДКБМЖПВВПЛЖКЛЯАНВРПИЛЖ',
 'ГНИРКДВСОМГМИРГОИВАТОЛХОМТСАООООДНОЕУАРЛОМММЬЧНМРСЕИСИПССНОРКЦОЗИДЫРНЕОККИБТНЯВОДАОБОЗЛТСДТСОРИПВЫЕЫММВХЗЛНСТООМНЫТХРЕЕОЛИТОАЕИОЬОАЬРЕНРТОИКЛДРОГАИБСКНДСНАЛМОЗОВСЕЛДОШИВТВЛАНТТЛВТДТТАСУ

Функция поиска относительного сдвига второй строки по максимальному индексу совпадения двух строк

In [144]:
def find_offset(first_string, second_string) -> int:
    offset, max_value = 0, 0.0
    for k in range(0, len(ALPHABET)):
        new_string = caesars_shift(second_string, ALPHABET, k)

        index = mutual_coincidence_index(first_string, new_string)
        if index > max_value:
            offset = k
            max_value = index

    return offset

Получаем относительный ключ

In [145]:
key = [0] * key_size

for i in range(1, len(strings)):
    key[i] = find_offset(strings[i - 1], strings[i])
    strings[i] = caesars_shift(strings[i], ALPHABET, key[i])

key

[0, 2, 25, 22, 29, 5, 19, 3, 12, 21, 10, 13, 7, 16, 9, 6, 4]

Функция расшифрования 

In [146]:
def decrypt(text: str, key: list[int], alphabet_chars: str = ALPHABET) -> str:
    decrypted_text = ""
    for i, char in enumerate(text):
        offset = key[i % len(key)]
        decrypted_text += caesars_shift(char, alphabet_chars, offset)

    return decrypted_text

Перебираем всевозможные сдвиги для ключа и ищем расшифрованный текст с наименьшим xi^2

In [147]:
min_xi2 = float("inf")
edit_key = None

for offset in range(len(ALPHABET)):
    new_key = [(offset + i) % len(ALPHABET) for i in key]
    plain_text = decrypt(text, new_key).lower()

    bigram_freq = bigram_frequencies(plain_text)
    xi2 = xi_square(bigram_freq, BIGRAM_FREQUENCIES)
    if xi2 < min_xi2:
        edit_key = new_key
        min_xi2 = xi2

edit_key

[8, 10, 3, 0, 7, 13, 27, 11, 20, 29, 18, 21, 15, 24, 17, 14, 12]

In [148]:
decrypt(text, edit_key)

'ТОРГОВЦЫПИСАТЕЛИИУЧЕНЫЕШАРЛАТАНЫИАФЕРИСТЫТЫСЯЧЕЛЕТИЯМИРАЗВИВАЛИНАУКУОСЕКРЕТАХДОВОДЯИХСОЗДАНИЕДОСОВЕРШЕНСТВАСЛУЖИЛИТАИНАМНАСЫЩАЛИСВОИПОТРЕБНОСТИВНИХБЕЗТАИННЕМОЖЕТБЫТЬНЕТОЛЬКОГОСУДАРСТВАНОДАЖЕМАЛОИОБЩНОСТИЛЮДЕИБЕЗНИХНЕЛЬЗЯВЫИГРАТЬСРАЖЕНИЕИЛИВЫГОДНОПРОДАТЬТОВАРОДОЛЕТЬСВОИХПОЛИТИЧЕСКИХПРОТИВНИКОВВЖЕСТОКОИБОРЬБЕЗАВЛАСТЬИЛИСОХРАНИТЬПЕРВЕНСТВОВТЕХНОЛОГИИТАИНЫСОСТАВЛЯЮТОСНОВУНАУКИТЕХНИКИИПОЛИТИКИЛЮБОИЧЕЛОВЕЧЕСКОИФОРМАЦИИЯВЛЯЯСЬЦЕМЕНТОМГОСУДАРСТВЕННОСТИИСТОРИЯХРАНИТТАКМНОГОСЕКРЕТОВЧТОПРОСТОУДИВИТЕЛЬНОДОЧЕГОЛЮДЯМОНИНЕОБХОДИМЫСЛУЖБАБЕЗОПАСНОСТИПЫТАЕТСЯДЕЛИТЬИХНАРЯДУРОВНЕИОТДЛЯСЛУЖЕБНОГОПОЛЬЗОВАНИЯДОСОВЕРШЕННОСЕКРЕТНОИСУГУБОДОВЕРИТЕЛЬНОАМЕРИКАНСКИИФИЗИКРИЧАРДФЕИНМАНШУТИЛЧТОПРИРАБОТЕНАДСОЗДАНИЕМАТОМНОИБОМБЫЕМУНАРЯДУСДОКУМЕНТАМИИМЕЮЩИМИПОМЕТКУТОЕСТЬБУКВАЛЬНОСЬЕСТЬПОСЛЕПРОЧТЕНИЯПОПАДАЛИСЬИНОГДАБУМАГИИСОШТАМПОМУНИЧТОЖИТЬДОПРОЧТЕНИЯСКОЛЬНИВЫСОКОНАУЧНАТЕОРИЯЛЕЖАЩАЯВОСНОВЕТАКОИКЛАССИФИКАЦИИОНАСВОДИТСЯКЗАУРЯДНОИДИСКРИМИНАЦИИГРУППЛЮДЕИНАРУШАЯИХЕСТЕСТВЕННЫЕПРАВАЕСЛИФИНАНСОВЫЕХИЩЕНИЯЮРИДИЧЕСКИМОЖНОДЕЛИТЬНАМ